In [1]:
#genereal code is used to implement model
# code referenced from: https://towardsdatascience.com/simple-transformers-introducing-the-easiest-bert-roberta-xlnet-and-xlm-library-58bf8c59b2a3
import pandas as pd
import numpy as np 

In [2]:
import wandb

In [ ]:
#to see the results on the wandb website use the authorization key: 9ebe3fdc95147616015f272bf80ff34c96b5389a
wandb.init(project="mehul-project")

In [3]:
data = pd.read_csv('tweet_emotions.csv')

In [4]:
data.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [5]:
data = data[["sentiment","content"]]

In [6]:
data.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [7]:
data['sentiment'] = data['sentiment'].apply(lambda x : x if x in ['happiness', 'sadness', 'worry', 'neutral'] else "other") 

In [8]:
data.head()

,sentiment,content
0,other,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,other,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [9]:
data= pd.DataFrame({
    'text': data['content'].replace(r'\n', ' ', regex=True),
    'label':data["sentiment"]
})
data.head()

,text,label
0,@tiffanylue i know i was listenin to bad habi...,other
1,Layin n bed with a headache ughhhh...waitin o...,sadness
2,Funeral ceremony...gloomy friday...,sadness
3,wants to hang out with friends SOON!,other
4,@dannycastillo We want to trade with someone w...,neutral


In [10]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
data['label'] = labelencoder.fit_transform(data[['label']].values)

/vol/hpc-software/software/Anaconda3/2020.07/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [11]:
data.head()

,text,label
0,@tiffanylue i know i was listenin to bad habi...,2
1,Layin n bed with a headache ughhhh...waitin o...,3
2,Funeral ceremony...gloomy friday...,3
3,wants to hang out with friends SOON!,2
4,@dannycastillo We want to trade with someone w...,1


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    40000 non-null  object
 1   label   40000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 625.1+ KB


In [13]:
#taking fractions of the dataset for the experiments
data = data.sample(frac=0.5, random_state=14)

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40000 entries, 34504 to 13656
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    40000 non-null  object
 1   label   40000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 937.5+ KB


In [15]:
from sklearn.model_selection import train_test_split
#splitting the dataset into train and test
train_val, test = train_test_split(data,test_size=0.2, random_state=2)

In [16]:
train_val.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32000 entries, 29580 to 3698
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    32000 non-null  object
 1   label   32000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 750.0+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8000 entries, 29242 to 25122
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    8000 non-null   object
 1   label   8000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 187.5+ KB


In [17]:
##splitting the dataset into train and validation
train, val = train_test_split(train_val,test_size=0.2, random_state=2)

In [18]:
train.info()
val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25600 entries, 15294 to 28849
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    25600 non-null  object
 1   label   25600 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 600.0+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6400 entries, 23319 to 735
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    6400 non-null   object
 1   label   6400 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 150.0+ KB


In [19]:
from simpletransformers.classification import ClassificationModel

In [20]:
#specifying hyperparameters
train_args = {
    'evaluate_during_training': True,
    'logging_steps': 10,
    'num_train_epochs': 2,
    'use_early_stopping': True,
    'early_stopping_patience': 3,
    'evaluate_during_training_steps': 10,
    'save_eval_checkpoints': False,
    'train_batch_size': 16,
    'eval_batch_size': 32,
    'learning_rate': 1e-5
    'overwrite_output_dir': True,


    'wandb_project': "mehul-project",
}

In [21]:
model_bert = ClassificationModel('bert', 'bert-base-uncased',num_labels=5,args = train_args)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [22]:
#training the model
model_bert.train_model(train,eval_df=val)

/user/HS400/tn00441/.local/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:585: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/25600 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

wandb: Currently logged in as: mehulsi (use `wandb login --relogin` to force relogin)


Running Epoch 0 of 2:   0%|          | 0/800 [00:00<?, ?it/s]

...

/vol/hpc-software/software/Anaconda3/2020.07/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


  0%|          | 0/6400 [00:00<?, ?it/s]

  0%|          | 0/6400 [00:00<?, ?it/s]

  0%|          | 0/6400 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
result, model_outputs, wrong_predictions = model_bert.eval_model(test)

In [ ]:
print(result)
print(model_outputs)